In [4]:
import requests
import pandas as pd

def download_file_from_google_drive(file_id, destination):
    """
    Downloads a file from Google Drive given its file ID.
    
    Parameters:
        file_id (str): The ID of the file on Google Drive.
        destination (str): The path where the file will be saved.
    """
    URL = "https://drive.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params={'id': file_id}, stream=True)
    
    # Check for confirmation token
    token = None
    for key, value in response.cookies.items():
        if key.startswith('confirm'):
            token = value
            break
    
    if token:
        response = session.get(URL, params={'id': file_id, 'confirm': token}, stream=True)
    
    # Write file to destination
    with open(destination, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    
    print(f"File downloaded and saved to {destination}")

# Google Drive file ID from the shared link
file_id = '1PCZAdl7t2wyIapwoBAIDlPDsEU0xhiJJ'
destination = 'downloaded_data_europe.csv'

# Download the file
download_file_from_google_drive(file_id, destination)

# Load the data into a DataFrame
df = pd.read_csv(destination)

# Display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Remove rows that are completely empty
df_cleaned = df.dropna(how='all')

# Display the cleaned DataFrame
display(df_cleaned)





File downloaded and saved to downloaded_data_europe.csv


,placeDcid,placeName,Date:Annual_Consumption_Electricity,Value:Annual_Consumption_Electricity,Source:Annual_Consumption_Electricity,Date:Annual_Loss_Electricity,Value:Annual_Loss_Electricity,Source:Annual_Loss_Electricity,Date:Annual_Emissions_CarbonDioxide_ElectricityGeneration,Value:Annual_Emissions_CarbonDioxide_ElectricityGeneration,Source:Annual_Emissions_CarbonDioxide_ElectricityGeneration,Date:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration,Value:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration,Source:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration,Date:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration,Value:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration,Source:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration
0,country/ALB,Albania,2019.0,6.179270e+09,https://unstats.un.org/unsd/energystats/data/,2019.0,1.081830e+09,https://unstats.un.org/unsd/energystats/data/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,country/AND,Andorra,2019.0,4.898400e+08,https://unstats.un.org/unsd/energystats/data/,2019.0,6.565000e+07,https://unstats.un.org/unsd/energystats/data/,2021.0,115000.0,https://api.climatetrace.org/,2021.0,115000.0,https://api.climatetrace.org/,2021.0,115000.0,https://api.climatetrace.org/
2,country/ANT,Netherlands Antilles,2011.0,9.890000e+08,https://unstats.un.org/unsd/energystats/data/,2011.0,2.120000e+08,https://unstats.un.org/unsd/energystats/data/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,country/ARM,Armenia,2018.0,5.402561e+09,https://unstats.un.org/unsd/energystats/data/,2018.0,6.114350e+08,https://unstats.un.org/unsd/energystats/data/,2021.0,1845000.0,https://api.climatetrace.org/,2021.0,1845000.0,https://api.climatetrace.org/,2021.0,1845000.0,https://api.climatetrace.org/
4,country/AUT,Austria,2018.0,6.307400e+10,https://unstats.un.org/unsd/energystats/data/,2018.0,3.459000e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,12158000.0,https://api.climatetrace.org/,2021.0,12158000.0,https://api.climatetrace.org/,2021.0,12158000.0,https://api.climatetrace.org/
5,country/AZE,Azerbaijan,2018.0,1.781150e+10,https://unstats.un.org/unsd/energystats/data/,2018.0,2.219800e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,16250000.0,https://api.climatetrace.org/,2021.0,16250000.0,https://api.climatetrace.org/,2021.0,16250000.0,https://api.climatetrace.org/
6,country/BEL,Belgium,2018.0,8.275500e+10,https://unstats.un.org/unsd/energystats/data/,2018.0,3.798000e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,18000000.0,https://api.climatetrace.org/,2021.0,18000000.0,https://api.climatetrace.org/,2021.0,18000000.0,https://api.climatetrace.org/
7,country/BGR,Bulgaria,2018.0,2.989500e+10,https://unstats.un.org/unsd/energystats/data/,2018.0,3.102000e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,22458000.0,https://api.climatetrace.org/,2021.0,22458000.0,https://api.climatetrace.org/,2021.0,22458000.0,https://api.climatetrace.org/
8,country/BIH,Bosnia and Herzegovina,2018.0,1.145600e+10,https://unstats.un.org/unsd/energystats/data/,2018.0,1.349000e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,9905000.0,https://api.climatetrace.org/,2021.0,9905000.0,https://api.climatetrace.org/,2021.0,9905000.0,https://api.climatetrace.org/
9,country/BLR,Belarus,2019.0,1.650450e+10,https://unstats.un.org/unsd/energystats/data/,2019.0,2.711000e+09,https://unstats.un.org/unsd/energystats/data/,2021.0,16212000.0,https://api.climatetrace.org/,2021.0,16212000.0,https://api.climatetrace.org/,2021.0,16212000.0,https://api.climatetrace.org/


import datacommons as dc
import pandas as pd

# Define the variables of interest
variables = [
    'Annual_Consumption_Electricity',
    'Annual_Loss_Electricity',
    'Annual_Emissions_CarbonDioxide_ElectricityGeneration',
    'Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration',
    'Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration'
]

# Get all European countries (place type: Country)
places = dc.get_places_in('country/Europe', 'Country')
print(f"Places: {places}")  # Debugging line to check the places

place_names = {}
for place in places:
    names = dc.get_property_values([place], 'name').get(place, [])
    if names:
        place_names[place] = names[0]
    else:
        place_names[place] = 'Unknown'

# Initialize an empty DataFrame to store the data
data = []

# Fetch the latest data for each country and variable
for place in places:
    place_data = {}
    place_data['placeDcid'] = place
    place_data['placeName'] = place_names[place]
    
    # Fetch the latest value for each variable
    for var in variables:
        try:
            series = dc.get_stat_series(place, var)
            print(f"Place: {place}, Variable: {var}, Series: {series}")
            
            # Get the latest available date and value
            if series:
                latest_date = max(series.keys())
                latest_value = series[latest_date]
                place_data[f"Date:{var}"] = latest_date
                place_data[f"Value:{var}"] = latest_value
                place_data[f"Source:{var}"] = "https://datacommons.org/tools/download#pt=Country&place=europe&sv=Annual_Consumption_Electricity__Annual_Loss_Electricity__Annual_Emissions_CarbonDioxide_ElectricityGeneration__Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration__Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration&dtType=LATEST&facets=%7B%7D"  # Adjust the source URL as needed
            else:
                place_data[f"Date:{var}"] = None
                place_data[f"Value:{var}"] = None
                place_data[f"Source:{var}"] = None
        except Exception as e:
            place_data[f"Date:{var}"] = None
            place_data[f"Value:{var}"] = None
            place_data[f"Source:{var}"] = None
            print(f"Error fetching data for {place} and variable {var}: {e}")
    
    print(f"Place data: {place_data}")
    data.append(place_data)

# Convert the data into a DataFrame
df = pd.DataFrame(data)

# Ensure the columns are in the desired order
desired_columns = [
    'placeDcid', 'placeName',
    'Date:Annual_Consumption_Electricity', 'Value:Annual_Consumption_Electricity', 'Source:Annual_Consumption_Electricity',
    'Date:Annual_Loss_Electricity', 'Value:Annual_Loss_Electricity', 'Source:Annual_Loss_Electricity',
    'Date:Annual_Emissions_CarbonDioxide_ElectricityGeneration', 'Value:Annual_Emissions_CarbonDioxide_ElectricityGeneration', 'Source:Annual_Emissions_CarbonDioxide_ElectricityGeneration',
    'Date:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration', 'Value:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration', 'Source:Annual_Emissions_CarbonDioxideEquivalent20YearGlobalWarmingPotential_ElectricityGeneration',
    'Date:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration', 'Value:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration', 'Source:Annual_Emissions_CarbonDioxideEquivalent100YearGlobalWarmingPotential_ElectricityGeneration'
]

df = df[desired_columns]

# Display the first few rows to verify the structure
print("Data retrieved from DataCommons:")
print(df.head())

# Clean the data by dropping rows with missing values
df_clean = df.dropna()

# Reset the index after cleaning
df_clean.reset_index(inplace=True, drop=True)

# Optionally, save the cleaned dataset to a CSV file
df_clean.to_csv('Cleaned_Europe_Energy_Data.csv', index=False)
print("Cleaned data saved as 'Cleaned_Europe_Energy_Data.csv'.")

# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)    # Show all rows
pd.set_option('display.max_columns', None) # Show all columns

# Display the entire cleaned DataFrame
print("Cleaned data:")
print(df_clean)




import requests

# Definir la URL de la API de Data Commons
url = 'https://api.datacommons.org/place?utm_medium=explore&mprop=count&p=country:USA'

# Realizar la solicitud GET
response = requests.get(url)

# Verificar el estado de la respuesta
if response.status_code == 200:
    # Convertir la respuesta JSON en un diccionario
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code}")
    